In [1]:
import pandas as pd

df = pd.read_csv("./data/subject-info.csv", sep=";", encoding="utf-8")
print(df.head())


  Patient ID  Follow-up period from enrollment (days)  days_4years  \
0      P0001                                     2065         1460   
1      P0002                                     2045         1460   
2      P0003                                     2044         1460   
3      P0004                                     2044         1460   
4      P0005                                     2043         1460   

   Exit of the study  Cause of death Age  Gender (male=1)  Weight (kg)  \
0                NaN               0  58                1           83   
1                NaN               0  58                1           74   
2                NaN               0  69                1           83   
3                NaN               0  56                0           84   
4                NaN               0  70                1           97   

   Height (cm) Body Mass Index (Kg/m2)  ...  \
0          163                    31,2  ...   
1          160                    28,9  

In [2]:
df.info()
df.shape
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Columns: 103 entries, Patient ID to Nitrovasodilator (yes=1)
dtypes: float64(30), int64(49), object(24)
memory usage: 798.4+ KB


Index(['Patient ID', 'Follow-up period from enrollment (days)', 'days_4years',
       'Exit of the study', 'Cause of death', 'Age', 'Gender (male=1)',
       'Weight (kg)', 'Height (cm)', 'Body Mass Index (Kg/m2)',
       ...
       'Angiotensin-II receptor blocker (yes=1)',
       'Anticoagulants/antitrombotics  (yes=1)', 'Betablockers (yes=1)',
       'Digoxin (yes=1)', 'Loop diuretics (yes=1)', 'Spironolactone (yes=1)',
       'Statins (yes=1)', 'Hidralazina (yes=1)', 'ACE inhibitor (yes=1)',
       'Nitrovasodilator (yes=1)'],
      dtype='object', length=103)

In [3]:
import pandas as pd
import numpy as np

def clean_data(df):
    print("\n🔹 Starte Datenbereinigung...")
    
    # 1. Entferne doppelte Spaltennamen, falls vorhanden
    df = df.loc[:, ~df.columns.duplicated()].copy()
    
    # 2. Identifiziere nicht-numerische Spalten
    problematic_columns = df.select_dtypes(include=['object']).columns.tolist()
    
    # 3. Entferne nicht-numerische Spalten aus der Konvertierungsliste
    exclude_columns = ["Patient_ID", "Holter_onset_hhmmss"]  # Falls weitere nötig sind, ergänzen
    convertible_columns = [col for col in problematic_columns if col not in exclude_columns]
    
    # 4. Ersetze Kommas durch Punkte und konvertiere zu Float
    for col in convertible_columns:
        df[col] = df[col].str.replace(',', '.').str.strip()
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Setzt nicht konvertierbare Werte auf NaN
    
    print(f"✅ Erfolgreich konvertierte Spalten: {convertible_columns}")
    print(f"⚠️ Nicht konvertierte Spalten (enthielten Text): {exclude_columns}")
    
    # 5. Fehlende Werte behandeln
    missing_values = df.isnull().sum()
    high_missing = missing_values[missing_values > 0.3 * len(df)].index.tolist()
    print(f"\n⚠️ Entferne Spalten mit >30% fehlenden Werten: {high_missing}")
    df.drop(columns=high_missing, inplace=True)
    df.fillna(df.median(), inplace=True)  # Verbleibende NaN mit Median ersetzen
    
    # 6. Spaltennamen bereinigen
    df.columns = df.columns.str.replace(r"[^A-Za-z0-9_]", "", regex=True)
    df.columns = df.columns.str.replace(r"_+", "_", regex=True).str.strip("_")
    print("✅ Spaltennamen bereinigt!")
    
    print("\n✅ Datenbereinigung abgeschlossen!")
    return df

# Bereinigte Daten abrufen
df = clean_data(df)


🔹 Starte Datenbereinigung...
✅ Erfolgreich konvertierte Spalten: ['Patient ID', 'Age', 'Body Mass Index (Kg/m2)', 'Albumin (g/L)', 'Normalized Troponin', 'Total Cholesterol (mmol/L)', 'Gamma-glutamil transpeptidase (IU/L)', 'Glucose (mmol/L)', 'Hemoglobin (g/L)', 'HDL (mmol/L)', 'Potassium (mEq/L)', 'LDL (mmol/L)', 'Protein (g/L)', 'T3 (pg/dL)', 'T4 (ng/L)', 'Troponin (ng/mL)', 'TSH (mIU/L)', 'Urea (mg/dL)', 'Cardiothoracic ratio', 'Holter onset (hh:mm:ss)', 'Number of ventricular premature contractions per hour', 'SDNN (ms)', 'RMSSD (ms)', 'pNN50 (%)']
⚠️ Nicht konvertierte Spalten (enthielten Text): ['Patient_ID', 'Holter_onset_hhmmss']

⚠️ Entferne Spalten mit >30% fehlenden Werten: ['Patient ID', 'Exit of the study', 'T3 (pg/dL)', 'Holter onset (hh:mm:ss)', 'Longest RR pause (ms)', 'Bradycardia', 'SDNN (ms)', 'SDANN (ms)', 'RMSSD (ms)', 'pNN50 (%)']
✅ Spaltennamen bereinigt!

✅ Datenbereinigung abgeschlossen!


In [ ]:
import pandas as pd
import numpy as np

# 🔹 Funktion zur vollständigen Datenbereinigung
def clean_data(df):
    print("\n🔷 Starte Datenbereinigung...")

    # 🔹 1. Entferne doppelte Spaltennamen, falls vorhanden
    df = df.loc[:, ~df.columns.duplicated()].copy()

    # 🔹 2. Falls 'Average_RR_ms' zwei Spalten enthält, korrekt aufteilen
    if "Average_RR_ms" in df.columns:
        if isinstance(df["Average_RR_ms"], pd.DataFrame):
            df["Average_RR_1_ms"] = df["Average_RR_ms"].iloc[:, 0]
            df["Average_RR_2_ms"] = df["Average_RR_ms"].iloc[:, 1]
            df.drop(columns=["Average_RR_ms"], inplace=True)
            print("✅ 'Average_RR_ms' erfolgreich aufgeteilt in zwei Spalten!")

    # 🔹 3. Problematische numerische Spalten bereinigen
    problematic_columns = df.select_dtypes(include=['object']).columns

    # 🔹 4. Entferne nicht-numerische Spalten aus der Konvertierungsliste
    exclude_columns = ["Patient_ID", "Holter_onset_hhmmss"]
    convertible_columns = [col for col in problematic_columns if col not in exclude_columns]

    # 🔹 5. Ersetze Kommas durch Punkte und konvertiere nur echte numerische Werte
    for col in convertible_columns:
        try:
            df[col] = df[col].str.replace(',', '.').astype(float)
        except ValueError:
            print(f"⚠️ Spalte {col} konnte nicht konvertiert werden und bleibt im Originalformat.")

    print(f"✅ Erfolgreich konvertierte Spalten: {convertible_columns}")
    print(f"⚠️ Nicht konvertierte Spalten (enthielten Text): {exclude_columns}")

    # 🔹 6. Fehlende Werte behandeln
    missing_values = df.isnull().sum()
    high_missing = missing_values[missing_values > 0.3 * len(df)].index.tolist()
    print(f"⚠️ Entferne Spalten mit >30% fehlenden Werten: {high_missing}")
    df.drop(columns=high_missing, inplace=True)
    df.fillna(df.median(numeric_only=True), inplace=True)

    # 🔹 7. Spaltennamen bereinigen
    df.columns = (df.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)  # Ersetze Leerzeichen durch Unterstriche
        .str.replace(r"[^A-Za-z0-9_]", "", regex=True)  # Entferne Sonderzeichen
        .str.replace(r"_+", "_", regex=True)  # Ersetze mehrere Unterstriche durch einen
    )

    print("✅ Spaltennamen bereinigt!")

    print("\n✅ Datenbereinigung abgeschlossen!")
    return df

# 🔹 Bereinigte Daten abrufen
df = clean_data(df)